# Слабо структурированные данные сквозь призму Haskell

## Кто я?

- Валявин Кирилл
- Haskell разработчик (движок Тинькофф Путешествий)
- Пишу на Haskell с 2018-го года

## Валидация данных
  - Это антипаттерн!
  - [Parse, don't validate](https://lexi-lambda.github.io/blog/2019/11/05/parse-don-t-validate/)
  - Типы в Haskell хороши для моделирования
  - Для JSON есть пакет [aeson](https://hackage.haskell.org/package/aeson)
  - Но всегда ли удобно парсить?


## Слабо структурированные данные
  - [No, dynamic type systems are not inherently more open](https://lexi-lambda.github.io/blog/2020/01/19/no-dynamic-type-systems-are-not-inherently-more-open/)
  - Иногда удобнее работать с сырыми данными
  - Но как это делать в Haskell?
  - Оптика!

## Оптика?
* Композабельные первоклассные аксессоры
* Позволяют проходить через рекорды, варианты, коллекции и т. д.
* Подробнее в книге Криса Пеннера [Optics By Example](https://leanpub.com/optics-by-example)
* Для JSON есть пакет [lens-aeson](https://hackage.haskell.org/package/lens-aeson)

## Подготовка среды

In [3]:
:set -XOverloadedStrings -XQuasiQuotes
:set -XFlexibleContexts -XTypeApplications

In [4]:
import Control.Lens
import Data.Aeson
import Data.Aeson.Lens

import qualified Data.ByteString as BS
import Chrono.TimeStamp

## Поехали!

In [5]:
y42 = 
  "{\"x\":0}" & _Object . at "y" .~ Just (Integer 42)
y42
y42 & _Object . at "x" .~ Nothing

Line 2: Use ?~
Found:
_Object . at "y" .~ Just (Integer 42)
Why not:
((_Object . at "y") ?~ Integer 42)

"{\"x\":0,\"y\":42}"

"{\"y\":42}"

In [6]:
makeReadable bs = -- IHaskell красиво печатает Value
  bs ^?! _JSON @_ @Value

makeReadable $
  "{}" & _Object . at "x" . below _Integer ?~ 42

{
    "x": 42
}

#### Реальные данные (из "Optics By Example")

In [8]:
pods <- BS.readFile "pods.json"
pods & makeReadable

{
    "apiVersion": "v1",
    "items": [
        {
            "apiVersion": "v1",
            "kind": "Pod",
            "metadata": {
                "creationTimestamp": "2019-03-23T19:42:21Z",
                "labels": {
                    "name": "redis",
                    "region": "usa"
                },
                "name": "redis-h315w"
            },
            "spec": {
                "containers": [
                    {
                        "image": "redis",
                        "name": "redis",
                        "ports": [
                            {
                                "containerPort": 27017,
                                "hostPort": 27017,
                                "name": "redis",
                                "protocol": "TCP"
                            }
                        ],
                        "resources": {
                            "requests": {
                                "cpu": "100m"
                

In [9]:
pods ^.. key "items" . values . key "kind" . _String

["Pod","Pod"]

#### Но теперь мы снова хотим парсить

In [10]:
items = key "items" . values

pods ^? items
      . key "metadata" 
      . key "creationTimestamp"
      . _JSON @_ @TimeStamp

Just 2019-03-23T19:42:21.000000000Z

In [11]:
metadataTimestamp =
    key "metadata" 
  . key "creationTimestamp"
  . _JSON @_ @TimeStamp

mar19 = 
  read @TimeStamp "2019-03-00T00:00:00Z"

pods ^.. items
       . filteredBy ( metadataTimestamp
                    . to (compare mar19)
                    . only LT) -- те, что созданы до 19 марта
       . key "metadata"

{
    "creationTimestamp": "2019-03-23T19:42:21Z",
    "labels": {
        "name": "redis",
        "region": "usa"
    },
    "name": "redis-h315w"
}

#### Индексы

In [13]:
let xy = "{ \"x\": 0, \"y\": 42}"

makeReadable $ -- ничего не меняем, но зависим от индекса
  xy & members %@~ \i v -> v

xy ^@.. members . reindexed id selfIndex

{
    "x": 0,
    "y": 42
}

[(Number 0.0,Number 0.0),(Number 42.0,Number 42.0)]

In [ ]:
metadataRegion =
    key "metadata"
  . key "labels"
  . key "region"
  . _String

withRegion =
  pods & items
       . reindexed ( view metadataRegion ) selfIndex
      <. key "metadata" . key "labels" . key "name" . _String
     %@~ \region name -> name <> "-" <> region
     
withRegion ^.. items . key "metadata" . key "labels"

{
    "name": "redis-usa",
    "region": "usa"
}

{
    "name": "web-usa",
    "region": "usa"
}

## Спасибо за внимание!
### Вопросы?